## 2. Типы данных в PostgreSQL

SQL — это язык со строгой типизацией, в котором каждый элемент данных имеет некоторый тип, определяющий его поведение и допустимое использование.

числовые типы — для хранения чисел (целых и дробных);
типы даты/времени — для хранения даты, времени, часовых поясов;
символьные типы — для хранения символов или строк;
логический тип — для хранения значений типа «истина», «ложь».

## 3. Даты: основные типы

Timestamp — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

SELECT NOW()
select CURRENT_TIMESTAMP

Timestamp with time zone позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

show timezone

select now() at time zone 'Europe/Moscow'

Задание 3.1
Напишите запрос, который выведет текущие время и дату в часовом поясе Лос-Анджелеса ("America/Los_Angeles").
Столбец в выдаче — now (время и дата в нужном часовом поясе).

select now() at time zone 'America/Los_Angeles'

Для получения текущей даты можно использовать
select now()::date
или 
select CURRENT_DATE

Задание 3.2
Предположим, у нас есть дата и время какого-то события и мы хотим посмотреть, к какой дате оно относится для Москвы и для UTC.
Используйте подзапрос
with x as 
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)
и выведите дату в ts в Московском часовом поясе и в поясе UTC. Столбцы в выдаче: dt_msk (дата в московском часовом поясе), dt_utc (дата в UTC).


with x as 
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)
select
  (ts at time zone 'Europe/Moscow')::date dt_msk,
        (ts)::date dt_utc
from x

Interval — тип данных, позволяющий хранить разницу между двумя временными метками. 

## 4. Функции и операторы для работы с датами

                                    Функция extract() получает из значений даты/времени такие поля, как год или час.

SELECT EXTRACT(DAY FROM TIMESTAMP '2001-02-16 20:38:40');
SELECT EXTRACT(DAY FROM INTERVAL '40 days 1 minute');

SELECT EXTRACT(HOUR FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(MONTH FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(YEAR FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(ISOYEAR FROM DATE '2006-01-01');

SELECT EXTRACT(ISOYEAR FROM DATE '2006-01-02');

SELECT EXTRACT(WEEK FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(CENTURY FROM TIMESTAMP '2000-12-16 12:21:13');
SELECT EXTRACT(CENTURY FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(DECADE FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(EPOCH FROM TIMESTAMP WITH TIME ZONE '2001-02-16 20:38:40.12-08');
SELECT EXTRACT(EPOCH FROM INTERVAL '5 days 3 hours');

SELECT to_timestamp(982384720.12);

SELECT EXTRACT(DOW FROM TIMESTAMP '2001-02-16 20:38:40');

SELECT EXTRACT(DOY FROM TIMESTAMP '2001-02-16 20:38:40');

Задание 4.1
Напишите запрос, который выведет год, месяц и количество доставок.
Отсортируйте по году и по месяцу в порядке возрастания.
Столбцы в выдаче: year_n (номер года), month_n (номер месяца), qty (количество доставок).

select 
    extract(year from s.ship_date) year_n,
    extract(month from s.ship_date) month_n,
    count(s.ship_id) qty
from sql.shipment s
group by 1,2
order by 1,2

                            Функция to_char() нужна для форматирования даты времени и интервалов в нужный текст.

Функция	                        Тип результата	Описание	                     Пример
to_char(timestamp[date],text)	text	        преобразует время в текст	    to_char(current_timestamp, 'HH12:MI:SS')
to_char(interval, text)	        text	        преобразует интервал в текст	to_char(interval '15h 2m 12s', 'HH24:MI:SS')

Код форматирования	        Описание
HH24	                    час (00-23)
MI	                        минута (00-59)
SS	                        секунда (00-59)
MS	                        миллисекунда (000-999)
SSSS	                    число секунд с начала суток (0-86399)
YYYY	                    год (4 или более цифр)
YY	                        последние две цифры года
month	                    полное название месяца в нижнем регистре (дополненное пробелами до девяти символов)
MM	                        номер месяца (01-12)
day	                        полное название дня недели в нижнем регистре, дополненное пробелами до девяти символов
DDD	                        номер дня в году (001-366)
DD	                        день месяца (01-31)
ID	                        номер дня недели по ISO 8601, считая с понедельника (1) до воскресенья (7)
WW                         	номер недели в году (1-53); первая неделя начинается в первый день года

select to_char(now(),'"Hello! Today is" DAY yyyy-Mon-dd')

Задание 4.2
Давайте выведем текст текущего времени для сервиса точного времени.
Напишите запрос, который выводит текст "Точное время x часов y минут z секунд" (текст в кавычки заключать не нужно), где x, y, z — часы, минуты и секунды соответственно, при условии, что сообщение нужно вывести для московского часового пояса.
Время введите в 24-часовом формате. Столбцы в выдаче: msg (сообщение).

select to_char(now() at time zone 'Europe/Moscow','Точное время HH24 часов MI минут SS секунд') msg

                        Функция date_trunc() позволяет отсечь заданное время, дату или дату со временем до нужной точности.

select date_trunc('minute',now())

Для получения разной степени точности вместо minute можно использовать следующие параметры:
microseconds;
milliseconds;
second;
minute;
hour;
day;
week;
month;
quarter;
year;
decade;
century;
millennium.

Задание 4.3
Напишите запрос, который выведет дату доставки, округлённую до квартала, и общую массу доставок.
Отсортируйте по кварталу в порядке возрастания.
Столбцы в выдаче: q (начало квартала, тип date), total_weight (сумма масс доставок за квартал).

select 
    date_trunc('quarter', s.ship_date)::date q,
    sum(s.weight) total_weight
from sql.shipment s
group by 1
order by 1

                                                            МАТЕМАТИЧЕСКИЕ ОПЕРАТОРЫ

select '2019-01-01'::date + 10
select '2019-01-01'::date + 500
select '2019-02-10'::date - '2017-03-01'::date

Задание 4.4
Напишите запрос, который выведет разницу между последним и первым днём доставки по каждому городу.
Отсортируйте по первому и второму столбцам. Столбцы в выдаче: city_name (название города) и days_active (время от первой до последней доставки в днях).

select 
    c.city_name,
    max(s.ship_date)-min(s.ship_date) days_active
from sql.shipment s
    join sql.city c on s.city_id = c.city_id
group by 1
order by 1, 2

## 5. Строковые данные: основные типы

CHARACTER - Cтрока фиксированной длины, дополненная пробелами.

CHARACTER VARYING - Строка ограниченной переменной длины.

TEXT - Cтрока неограниченной длины

## 6. Функции и операторы для работы со строками

Важно! Результатом соединения любых типов строковых данных будет тип text.

select 'select * from '||t.table_schema||'.'||t.table_name||';' query
from information_schema.tables t
where table_schema = 'shipping'

Важно! Если вы соединяете любую строку и NULL, то результатом будет NULL. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует NULL, используйте оператор coalesce.

Задание 6.1
Составим текстовый шаблон сообщения о доставке по конкретному водителю для наших клиентов.
Напишите SQL-запрос, который выведет следующее сообщение для каждого водителя по форме:
Ваш заказ доставит водитель #Имя Фамилия#. Его контактный номер: #Номер#, где #Имя Фамилия# и #Номер# взяты из справочника водителей.
Если номер не указан, то выведите прочерк (-). Для номеров рекомендуем использовать coalesce.
Пример из таблицы для наглядности: Ваш заказ доставит водитель Adel Al-Alawi. Его контактный номер: (901) 947-4433
Столбец к выдаче — msg (текст сообщения).

Вариант 1:
select 'Ваш заказ доставит водитель '||d.first_name||' '||d.last_name||'. Его контактный номер: '||coalesce(d.phone) msg
from sql.driver d
where d.phone is not null
union all
select 'Ваш заказ доставит водитель '||dd.first_name||' '||dd.last_name||'. Его контактный номер: -' msg
from sql.driver dd
where dd.phone is null

Вариант 2:
SELECT 
        'Ваш заказ доставит водитель '||d.first_name||' '||d.last_name||'. '||'Его контактный номер: '||coalesce(d.phone, '-') msg
FROM sql.driver d

Функции upper(your_text) и lower(your_text) переводят каждый символ вашего текста в верхний и нижний регистр соответственно.

select upper('Abc') s1 ,lower('xYz') s2

Задание 6.2
Cоставим справочник названий клиентов, у которых более десяти доставок. Данные сохраним в нижнем регистре, чтобы передавать их в другие системы (например, для обзвона), которые не чувствительны к регистру.
Напишите запрос, который выводит все id названий клиентов, у которых более десяти доставок, в нижнем регистре. Отсортируйте результат по cust_id в порядке возрастания. Столбцы в выдаче: cust_id (id клиента) и cust_name (название клиента в нижнем регистре).

select 
    c.cust_id cust_id,
    lower(c.cust_name) cust_name
from sql.customer c
    join sql.shipment s on c.cust_id = s.cust_id
group by 1
having count(s.ship_id) > 10
order by 1

С помощью функции replace() можно заменять символы в строках.
replace(string text, from text, to text)

select replace('малако','а','о')

Например, сделаем из строки "Hello, world!" строку "Hello!".
select replace('Hello, world!',', world','')

Задание 6.3
Составим справочник utm-меток, для того чтобы передавать город и штат прямо в адресной строке. (Если вы не знаете, что такое utm-метка, почитайте статью на Вики. К программе курса это не относится, но знать полезно.)
Напишите SQL-запрос, который выведет список сочетаний из справочника следующего вида: название_штата__название_города, где названия штата и города взяты из справочника городов и переведены в нижний регистр.
Столбец к выдаче — utm (форматированный штат-город).
Отсортируйте полученный справочник по алфавиту.
Обратите внимание! Все пробелы в названиях городов и штатов замените символом '_' (одно нижнее подчёркивание), а для разделения названий города и штата используйте '__' (два последовательных нижних подчёркивания).
Пример из таблицы для наглядности: new_jersey__union_city

select 
    replace(lower(c.state), ' ', '_')||'__'||replace(lower(c.city_name),' ', '_') utm
from sql.city c
order by 1

LEFT() И RIGHT()

with t as
(
select 'Один два три'::text sample_string
)
select 
 left(t.sample_string,4) one,
 right(left(t.sample_string,8),3) two,
 right(t.sample_string,3) three
from t

select left('0123456789', - 2), right('0123456789', - 2)

Задание 6.4
Представим, что к вам пришёл разработчик, который хочет сократить поле state в таблице city до четырёх символов, и попросил проверить, останутся ли значения в нём уникальными.
Чтобы ответить на этот вопрос, напишите SQL-запрос, который выведет первые четыре символа названия штата, и количество уникальных названий штатов, которому они соотвествуют. Оставьте только те, которые относятся к двум и более штатам.
Добавьте сортировку по первому столбцу. Столбцы в выдаче: code (четыре первых буквы штата), qty (количество уникальных названий штата, начинаюшихся на эти буквы).

select
    left(c.state, 4) code,
    count(distinct c.state) qty
from sql.city c
group by 1
having count(distinct c.state) >=2
order by 1

Функция format() используется для составления форматированного текста с подстановками. То же самое можно сделать через конкатенацию строк, но это неудобно и громоздко.

format(formatstr text [, argument1 text,argument2 text...])

select format('Hello, %s!', d.first_name) from shipping.driver d

Если в вашем шаблоне присутствует одинарная кавычка, то для удобства можно вместо одинарных кавычек использовать $$ (два знака доллара):
select $$ some_string with quotes ' $$

Задание 6.5
Давайте подготовим географическую сводку для каждого города.
Напишите SQL-запрос, который выведет описание региона в следующем формате: [city_name] is located in [state]. There's [population] people living there. Its area is [area] (обратите внимание, точку в конце ставить не нужно).
Отсортируйте по названию города в алфавитном порядке.
Столбец к выдаче — str (сводка).
Пример: Abilene is located in Texas. There's 115930 people living there. Its area is 105.10

select 
    format($$%s is located in %s. There's %s people living there. Its area is %s$$, c.city_name, c.state, c.population, c.area) str
from sql.city c
order by c.city_name